In [177]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import pdb

In [102]:
files = [f'data/Asteroids/Asteroid {i}.csv' for i in range(1, 21)]
dfs = [pd.read_csv(file) for file in files]
for i, _ in enumerate(dfs):
    dfs[i]['Asteroid'] = i + 1
    dfs[i].dropna(inplace=True)
    dfs[i].drop(dfs[i][dfs[i]['BIT_DEPTH'] < 0].index, inplace=True)
    dfs[i].drop(dfs[i][dfs[i]['RATE_OF_PENETRATION'] < 0].index, inplace=True)

In [139]:
dfs[0]['DRILL_BIT_NAME'].unique()

array(['Buzz Drilldrin', 'AstroBit', 'ChallengDriller'], dtype=object)

In [119]:
def sort_key(e):
    return e[1]

In [132]:
ranges = [[i, df['BIT_DEPTH'].iloc[0], df['BIT_DEPTH'].iloc[-1]] for i, df in zip(range(20), dfs)]
ranges = sorted(ranges, key=sort_key)
ranges

[[9, 2.73, 19337.05],
 [5, 97.77, 15324.06],
 [6, 100.98, 17876.5],
 [4, 105.22, 14734.5],
 [19, 113.08, 22583.02],
 [8, 119.8, 11315.02],
 [17, 138.57, 19506.52],
 [18, 144.52, 18043.01],
 [10, 170.55, 21170.55],
 [7, 199.18, 19424.51],
 [14, 199.58, 13550.0],
 [15, 216.63, 17762.74],
 [1, 242.59, 13781.0],
 [3, 370.07, 19331.03],
 [13, 553.04, 20974.52],
 [2, 730.65, 18754.52],
 [0, 745.01, 20628.53],
 [11, 745.01, 20628.53],
 [16, 1252.71, 23160.56],
 [12, 1356.54, 10910.52]]

In [147]:
def ran_to_str(range):
    return str(range[0]) + '-' + str(range[1])

In [178]:
def calc_time(examine):
    time = 0
    for i, row in examine.iterrows():
        if i == 0: continue
        pdb.set_trace()
        delta_depth = row['BIT_DEPTH'] - examine.at[i-1, 'BIT_DEPTH']
        time += delta_depth / row['RATE_OF_PENETRATION']
    time += (examine['DRILL_BIT_ID'].max() - 1) * examine['BIT_DEPTH'].max() / 100 * 30 / 3600
    return time

In [180]:
from collections import defaultdict
range_len = 2000
calc_ranges = [[range_len*i, range_len*(i+1)] for i in range(int(20000 / range_len))]
bits = ['Apollo', 'AstroBit', 'Buzz Drilldrin', 'ChallengDriller']

keys = [ran_to_str(calc_range) for calc_range in calc_ranges]
bits_times = dict(zip(keys, [defaultdict(list) for i in range(len(keys))]))
bits_costs = dict(zip(keys, [defaultdict(list) for i in range(len(keys))]))
i=0
for calc_range in calc_ranges:
    for trial in dfs:
        examine = trial[trial['BIT_DEPTH'] > calc_range[0]]
        examine = examine[examine['BIT_DEPTH'] < calc_range[1]]
        if len(examine) == 0:
            continue
        bit = examine['DRILL_BIT_NAME'].value_counts().idxmax()
        examine = examine[examine['DRILL_BIT_NAME'] == bit]
        
        time = calc_time(examine)
        if i == 0: print(time)
        i+=1
        

> c:\users\kunhu\appdata\local\temp\ipykernel_31768\3599798184.py(6)calc_time()



BdbQuit: 

In [85]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

models = []
train_cols = ['BIT_DEPTH','RATE_OF_PENETRATION','HOOK_LOAD','DIFFERENTIAL_PRESSURE','WEIGHT_ON_BIT']
for i in range(1, 21):
    df_select = df[df['Asteroid'] == i]
    x_train, x_test, y_train, y_test = train_test_split(df_select[train_cols], df_select['DRILL_BIT_USED'])
    model = DecisionTreeClassifier()
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    models.append(model)
    print(metrics.classification_report(y_test, y_pred))

                   precision    recall  f1-score   support

       AstroBit 2       1.00      1.00      1.00      1720
 Buzz Drilldrin 1       1.00      1.00      1.00      1197
ChallengDriller 3       1.00      1.00      1.00       927
ChallengDriller 4       1.00      1.00      1.00       368

         accuracy                           1.00      4212
        macro avg       1.00      1.00      1.00      4212
     weighted avg       1.00      1.00      1.00      4212

              precision    recall  f1-score   support

    Apollo 1       1.00      1.00      1.00      1605
    Apollo 2       1.00      1.00      1.00       584

    accuracy                           1.00      2189
   macro avg       1.00      1.00      1.00      2189
weighted avg       1.00      1.00      1.00      2189

              precision    recall  f1-score   support

    Apollo 1       1.00      1.00      1.00       232
  AstroBit 2       1.00      1.00      1.00      1572
  AstroBit 3       1.00      1.00  

In [60]:
import pdb
import numpy as np

In [90]:
class predictor:
    def __init__(self): # Add training algorithm
        self.models = models
        self.key_inputs = train_cols

    def __len__(self):
        return len(self.models)

    def predict(self, inputs):
        model_use = self.models[inputs['Asteroid'] - 1]
        inputs = np.array(inputs)[:5].reshape(1, -1)
        return model_use.predict(inputs)

    def predict_proba(self, inputs):
        model_use = self.models[inputs['Asteroid'] - 1]
        inputs = np.array(inputs)[:5].reshape(1, -1)
        return model_use.predict_proba(inputs)

In [92]:
my_test = df.iloc[0]
for input in 
    my_test

BIT_DEPTH                          793.02
RATE_OF_PENETRATION                296.36
HOOK_LOAD                          104.47
DIFFERENTIAL_PRESSURE               70.92
WEIGHT_ON_BIT                        4.71
DRILL_BIT_ID                            1
DRILL_BIT_NAME             Buzz Drilldrin
Asteroid                                1
DRILL_BIT_USED           Buzz Drilldrin 1
Name: 48, dtype: object

In [91]:
Predictor = predictor()
import numpy as np
arr = np.array(df.iloc[0]).reshape(1, -1)
Predictor.predict_proba(df.iloc[100])

# x_test.iloc[0]

array([[0., 1., 0., 0.]])

In [74]:
df.describe()

,BIT_DEPTH,RATE_OF_PENETRATION,HOOK_LOAD,DIFFERENTIAL_PRESSURE,WEIGHT_ON_BIT,DRILL_BIT_ID,Asteroid
count,290247.000000,290247.000000,290247.000000,290247.000000,290247.000000,290247.000000,290247.000000
mean,9903.366464,255.221120,150.959011,698.997702,41.042261,2.190924,11.110730
std,5424.899041,196.804204,41.678055,310.709351,10.832484,1.261408,5.865579
min,4.250000,0.000000,0.000000,-5399.000000,0.000000,1.000000,1.000000
25%,5347.195000,119.900000,124.000000,530.000000,34.940000,1.000000,6.000000
50%,9557.020000,190.340000,157.800000,716.000000,42.290000,2.000000,11.000000
75%,14147.045000,331.600000,176.700000,879.000000,48.920000,3.000000,17.000000
max,23160.560000,3488.900000,412.900000,2042.000000,262.670000,7.000000,20.000000
